In [1]:
from lxml import objectify
from lxml import etree
import numpy as np
from auto_naparm import auto_naparm
import copy

In [2]:
naparm_path = r"F:\Data\jrowland\2019-04-05\naparm\2019-04-05_s-059_NAPARM_002"

an = auto_naparm(naparm_path=naparm_path)

['455.99999999999994', '455.99999999999994', '455.99999999999994', '455.99999999999994']
SLM connection established
First mask loaded
Ready to trigger
PV connection successful


In [3]:
inter_group_interval = an.inter_group_interval

spiral_duration = an.durations[0] 
spiral_repetitions = an.repetitions[0]
stim_length = int(spiral_duration) * int(spiral_repetitions)

In [4]:
xml_path = r"F:\Data\jrowland\2019-04-05\naparm\2019-04-05_s-059_NAPARM_002\2019-04-05_s-059_NAPARM_002.xml"
tree = etree.parse(xml_path)
root= tree.getroot()
markpoint_elems = root.findall('PVMarkPointElement')


for i,point in enumerate(markpoint_elems):
    
    #dont change the dummy
    if point.attrib['UncagingLaserPower'] == '0': 
        mute_point = False
    else:
        mute_point = True

    if mute_point:       
        point.attrib['TriggerFrequency'] = 'Never'
        galvo_elem_point = next(node for node in point.getiterator() if node.tag == 'PVGalvoPointElement')
        galvo_elem_point.attrib['InitialDelay'] = str(inter_group_interval - stim_length)
    
    
    slm_trigger = copy.deepcopy(point)
    
    slm_trigger.attrib['UncagingLaser'] = 'Trigger'
    slm_trigger.attrib['UncagingLaserPower'] = '5' #this is 1000 PV in weird xml speak
    slm_trigger.attrib['AsyncSyncFrequency'] = 'None'
    slm_trigger.attrib['Repetitions'] = '1'

    galvo_elem_slm = next(node for node in slm_trigger.getiterator() if node.tag == 'PVGalvoPointElement')

    galvo_elem_slm.attrib['InitialDelay'] = '0'
    galvo_elem_slm.attrib['Duration'] = '1'
    galvo_elem_slm.attrib['SpiralRevolutions'] = '1'
    
    
    
    print(etree.tostring(point))
    
    if i != len(markpoint_elems) - 1:
        parent = point.getparent()
        parent.insert(parent.index(point)+1, slm_trigger)
    
    

        
tree.write(r"F:\Data\jrowland\2019-04-05\naparm\2019-04-05_s-059_NAPARM_001\altered.xml")
        
        
    


b'<PVMarkPointElement Repetitions="1" UncagingLaser="Uncaging" UncagingLaserPower="0" TriggerFrequency="None" TriggerSelection="None" TriggerCount="1" AsyncSyncFrequency="None" VoltageOutputCategoryName="None" VoltageRecCategoryName="None" parameterSet="CurrentSettings"><PVGalvoPointElement InitialDelay="0" InterPointDelay="0" Duration="1" SpiralRevolutions="1" Points="Point 1" Indices="1"/></PVMarkPointElement>'
b'<PVMarkPointElement Repetitions="25" UncagingLaser="Uncaging" UncagingLaserPower="2.28" TriggerFrequency="Never" TriggerSelection="PFI1" TriggerCount="1" AsyncSyncFrequency="FirstRepetition" VoltageOutputCategoryName="JR" VoltageOutputExperimentName="5ms" VoltageRecCategoryName="None" parameterSet="CurrentSettings"><PVGalvoPointElement InitialDelay="750.0" InterPointDelay="0" Duration="10" SpiralRevolutions="3" Points="Point 1" Indices="1"/></PVMarkPointElement>'
b'<PVMarkPointElement Repetitions="25" UncagingLaser="Uncaging" UncagingLaserPower="2.28" TriggerFrequency="Never

In [5]:
1

1